In [1]:
import warnings
import pandas as pd
import numpy as np
import os
import librosa
import numpy as np
from scipy.io.wavfile import read
from python_speech_features import mfcc
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

In [2]:
sr = 16000 # sample rate (16kHz)
# parameters to tune
n_estimators_rf = 100
#mfcc parameters
nc_mfcc = 10 # number of chunks
n_mfcc = 10
top_db_trim = 20

First, we load the dev dataset

In [3]:
df_dev = pd.read_csv("dsl_data/development.csv",index_col=0)
df_dev

,path,speakerId,action,object,Self-reported fluency level,First Language spoken,Current language used for work/school,gender,ageRange
Id,,,,,,,,,
0,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/0a312...,2BqVo8kVB2Skwgyb,change language,none,advanced,English (United States),English (United States),female,22-40
1,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/0ee42...,2BqVo8kVB2Skwgyb,activate,music,advanced,English (United States),English (United States),female,22-40
2,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/1d9f3...,2BqVo8kVB2Skwgyb,deactivate,lights,advanced,English (United States),English (United States),female,22-40
3,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/269fc...,2BqVo8kVB2Skwgyb,increase,volume,advanced,English (United States),English (United States),female,22-40
4,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/5bbda...,2BqVo8kVB2Skwgyb,increase,volume,advanced,English (United States),English (United States),female,22-40
...,...,...,...,...,...,...,...,...,...
9849,dsl_data/audio/speakers/vnljypgejkINbBAY/4fb3d...,vnljypgejkINbBAY,decrease,volume,native,English (United States),English (United States),male,22-40
9850,dsl_data/audio/speakers/vnljypgejkINbBAY/59e6a...,vnljypgejkINbBAY,deactivate,lights,native,English (United States),English (United States),male,22-40
9851,dsl_data/audio/speakers/vnljypgejkINbBAY/5c81c...,vnljypgejkINbBAY,deactivate,lights,native,English (United States),English (United States),male,22-40


In [4]:
df_dev.columns

Index(['path', 'speakerId', 'action', 'object', 'Self-reported fluency level ',
       'First Language spoken', 'Current language used for work/school',
       'gender', 'ageRange'],
      dtype='object')

Then we load the eval dataset

In [5]:
df_eval = pd.read_csv("dsl_data/evaluation.csv")
df_eval['Id'] = df_eval['Id'].apply(lambda x: x + 9854)
df_eval = df_eval.set_index('Id')
df_eval.head()

,path,speakerId,Self-reported fluency level,First Language spoken,Current language used for work/school,gender,ageRange
Id,,,,,,,
9854,dsl_data/audio/speakers/NgQEvO2x7Vh3xy2xz/f53c...,NgQEvO2x7Vh3xy2xz,native,English (United States),English (United States),male,22-40
9855,dsl_data/audio/speakers/k5bqyxx2lzIbrlg9/1d5f8...,k5bqyxx2lzIbrlg9,native,English (United States),English (United States),male,22-40
9856,dsl_data/audio/speakers/7B4XmNppyrCK977p/1c0d5...,7B4XmNppyrCK977p,native,English (United States),English (United States),male,41-65
9857,dsl_data/audio/speakers/k5bqyxx2lzIbrlg9/275c3...,k5bqyxx2lzIbrlg9,native,English (United States),English (United States),male,22-40
9858,dsl_data/audio/speakers/V4ZbwLm9G5irobWn/b7c7a...,V4ZbwLm9G5irobWn,native,English (United States),English (United States),female,41-65


In [6]:
df = pd.concat([df_dev, df_eval], sort=False)
mask = ~df["action"].isna()
print('Shape : ',df.shape)
df.head() 

Shape :  (11309, 9)


,path,speakerId,action,object,Self-reported fluency level,First Language spoken,Current language used for work/school,gender,ageRange
Id,,,,,,,,,
0,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/0a312...,2BqVo8kVB2Skwgyb,change language,none,advanced,English (United States),English (United States),female,22-40
1,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/0ee42...,2BqVo8kVB2Skwgyb,activate,music,advanced,English (United States),English (United States),female,22-40
2,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/1d9f3...,2BqVo8kVB2Skwgyb,deactivate,lights,advanced,English (United States),English (United States),female,22-40
3,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/269fc...,2BqVo8kVB2Skwgyb,increase,volume,advanced,English (United States),English (United States),female,22-40
4,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/5bbda...,2BqVo8kVB2Skwgyb,increase,volume,advanced,English (United States),English (United States),female,22-40


In [7]:
y_dev=np.array(df[mask]['action']+df[mask]['object']).T

In [8]:
paths = df['path']
df = df.drop(columns=['path'])

In [9]:
df = df.drop(columns=['speakerId', 'action', 'object', 'Self-reported fluency level ','First Language spoken', 'Current language used for work/school','gender', 'ageRange'])

### Audio features

In [10]:
def extract_features_mfcc(audio, num_chunks, num_coef=nc_mfcc):
    features = []
    for i in range(len(audio)):
        mfcc = librosa.feature.mfcc(y=audio[i], n_mfcc=num_coef)
        chunks = np.array_split(mfcc, num_chunks, axis=1)
        means = [np.mean(chunk, axis=1) for chunk in chunks]
        features.append(np.array(means).T)
    return features

In [11]:
def extract_feature_zcr(data):
    zcr = librosa.zero_crossings(data).sum() #.916 version
    return zcr

In [12]:
def extract_feature_onset(data):
    onset_frames = librosa.onset.onset_detect(y=data, sr=sr)
    onset_mean = onset_frames.mean()
    onset_std = onset_frames.std()
    return onset_mean,onset_std

In [13]:
def extract_feature_centroid(data):
    spect_cent = librosa.feature.spectral_centroid(y=data, sr=sr)
    spect_cent_mean = spect_cent.mean()
    spect_cent_std = spect_cent.std()
    return spect_cent_mean, spect_cent_std

In [14]:
def extract_duration(data):
    duration = librosa.get_duration(y=data, sr=sr)
    return duration

In [15]:
# arrays to store values for columns
durations = np.array([])
num_frames_arr = np.array([])
zcr_col = np.array([])
onset_mean = np.array([])
onset_std = np.array([])
spect_cent_mean = np.array([])
spect_cent_std = np.array([])
tempos = np.array([])

sample_rate = 16000
arr_data = []
for index, path in paths.items():
    data, _ = librosa.load(path,sr = sr, mono=True)
    
    # trim silence
    data, _ = librosa.effects.trim(data, top_db=top_db_trim)
    arr_data.append(data)
    
    #duration
    durations = np.append(durations,extract_duration(data))
    
    # zero crossing rate
    zcr_col = np.append(zcr_col,extract_feature_zcr(data))
    
    #onset frames
    ret_1,ret_2 = extract_feature_onset(data)
    onset_mean = np.append(onset_mean, ret_1)
    onset_std = np.append(onset_std, ret_2)
    
    #spectral centroids
    ret_1,ret_2 = extract_feature_centroid(data)
    spect_cent_mean = np.append(spect_cent_mean,ret_1)
    spect_cent_std = np.append(spect_cent_std,ret_2)

# mfcc features
features_mfcc = extract_features_mfcc(arr_data,nc_mfcc)

In [16]:
df['zcr'] = zcr_col
df['duration'] = durations
df['onset mean'] = onset_mean
df['onset std'] = onset_std
df['spect cent mean'] = spect_cent_mean
df['spect cent std'] = spect_cent_std
df.head()

,zcr,duration,onset mean,onset std,spect cent mean,spect cent std
Id,,,,,,
0,4070.0,1.184,17.50,10.190682,2602.715534,1213.057802
1,1603.0,0.704,3.50,0.500000,1827.121264,1449.153061
2,3282.0,1.280,17.60,10.809255,1628.021457,1449.529701
3,1369.0,0.992,18.20,8.611620,1397.768061,433.342282
4,1127.0,1.472,23.25,15.213070,960.330088,640.262653


We normalize these extracted features in [-1,1]

In [17]:
scaler = StandardScaler()

In [18]:
df['zcr'] = scaler.fit_transform(df[['zcr']].values.reshape(-1,1))
df['duration'] = scaler.fit_transform(df[['duration']].values.reshape(-1,1))
df['onset mean'] = scaler.fit_transform(df[['onset mean']].values.reshape(-1,1))
df['onset std'] = scaler.fit_transform(df[['onset std']].values.reshape(-1,1))
df['spect cent mean'] = scaler.fit_transform(df[['spect cent mean']].values.reshape(-1,1))
df['spect cent std'] = scaler.fit_transform(df[['spect cent std']].values.reshape(-1,1))
df.head()

,zcr,duration,onset mean,onset std,spect cent mean,spect cent std
Id,,,,,,
0,1.058822,0.246178,0.279698,0.314019,1.688605,1.004393
1,-0.510026,-0.824465,-1.666499,-1.853252,0.322227,1.576320
2,0.557706,0.460306,0.293599,0.452360,-0.028530,1.577233
3,-0.658835,-0.182079,0.377008,-0.039130,-0.434172,-0.884421
4,-0.812731,0.888564,1.079029,1.437251,-1.204813,-0.383169


In [19]:
a_mean = np.array(features_mfcc).reshape(len(df),nc_mfcc*n_mfcc)

In [20]:
df_features_mfcc = pd.DataFrame(a_mean).fillna(0)
df_features_mfcc.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-348.551849,-319.335999,-377.675140,-301.194153,-298.603455,-330.142761,-389.953430,-372.765442,-466.906097,-519.792969,...,3.022222,-10.588423,-10.499366,-10.401047,-26.580713,-36.643181,-7.548432,3.563060,6.830827,6.171805
1,-483.128418,-369.978973,-365.020355,-346.901306,-337.804688,-350.885620,-456.129333,-558.102295,-605.531250,-598.971252,...,-19.171438,-7.772737,-2.552499,6.840453,-2.082320,-22.186264,-17.359638,-7.877681,-19.985538,-8.465648
2,-601.203674,-603.713501,-507.916016,-530.811829,-534.048218,-480.279846,-528.254272,-633.983154,-338.566589,-468.089478,...,-23.722355,-10.011897,-8.519898,-10.439441,-26.227602,-24.089558,1.132828,-10.049913,-30.885525,9.953860
3,-373.138489,-311.546814,-368.780762,-413.504303,-369.568451,-277.892090,-562.955627,-431.036621,-15.616693,-425.439819,...,-18.998680,-28.718588,-15.661521,-13.194974,-14.598924,-20.866451,-7.213551,-3.183887,-2.013952,-8.955941
4,-548.048462,-563.210327,-532.331543,-510.005676,-509.754211,-499.770905,-588.408691,-766.812744,-611.130920,-582.894470,...,-24.033184,-31.483419,-33.698963,-21.212469,-17.651859,-13.166054,-8.405053,2.083387,2.366938,-3.876985


In [21]:
for col in df_features_mfcc.columns:
    df_features_mfcc[col] = (df_features_mfcc[col] - df_features_mfcc[col].min()) / (df_features_mfcc[col].max() - df_features_mfcc[col].min()) * 2 - 1
df_features_mfcc.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.245824,0.276959,0.155063,0.285123,0.291607,0.216640,0.080443,0.070499,-0.064845,-0.043809,...,0.241860,0.108583,-0.055494,-0.158023,-0.415336,-0.456717,-0.071031,0.165850,0.284754,0.284823
1,-0.037229,0.178903,0.180355,0.188964,0.211049,0.175661,-0.053831,-0.299620,-0.339699,-0.202808,...,-0.162842,0.159916,0.076653,0.109214,-0.020993,-0.220496,-0.222926,-0.000224,-0.127786,0.017004
2,-0.285576,-0.273658,-0.105233,-0.197947,-0.192229,-0.079971,-0.200177,-0.451155,0.189615,0.060018,...,-0.245828,0.119094,-0.022578,-0.158618,-0.409653,-0.251595,0.063370,-0.031756,-0.295470,0.354022
3,0.194111,0.292040,0.172839,0.048845,0.145775,0.319867,-0.270588,-0.045869,0.829932,0.145664,...,-0.159691,-0.221949,-0.141334,-0.201327,-0.222469,-0.198931,-0.065847,0.067911,0.148687,0.008033
4,-0.173775,-0.195235,-0.154029,-0.154175,-0.142305,-0.118477,-0.322233,-0.716416,-0.350801,-0.170524,...,-0.251496,-0.272355,-0.441275,-0.325595,-0.271611,-0.073109,-0.084293,0.144371,0.216082,0.100962


In [22]:
df = pd.merge(df, df_features_mfcc, left_index=True, right_index=True)
df.head()

,zcr,duration,onset mean,onset std,spect cent mean,spect cent std,0,1,2,3,...,90,91,92,93,94,95,96,97,98,99
Id,,,,,,,,,,,,,,,,,,,,,
0,1.058822,0.246178,0.279698,0.314019,1.688605,1.004393,0.245824,0.276959,0.155063,0.285123,...,0.241860,0.108583,-0.055494,-0.158023,-0.415336,-0.456717,-0.071031,0.165850,0.284754,0.284823
1,-0.510026,-0.824465,-1.666499,-1.853252,0.322227,1.576320,-0.037229,0.178903,0.180355,0.188964,...,-0.162842,0.159916,0.076653,0.109214,-0.020993,-0.220496,-0.222926,-0.000224,-0.127786,0.017004
2,0.557706,0.460306,0.293599,0.452360,-0.028530,1.577233,-0.285576,-0.273658,-0.105233,-0.197947,...,-0.245828,0.119094,-0.022578,-0.158618,-0.409653,-0.251595,0.063370,-0.031756,-0.295470,0.354022
3,-0.658835,-0.182079,0.377008,-0.039130,-0.434172,-0.884421,0.194111,0.292040,0.172839,0.048845,...,-0.159691,-0.221949,-0.141334,-0.201327,-0.222469,-0.198931,-0.065847,0.067911,0.148687,0.008033
4,-0.812731,0.888564,1.079029,1.437251,-1.204813,-0.383169,-0.173775,-0.195235,-0.154029,-0.154175,...,-0.251496,-0.272355,-0.441275,-0.325595,-0.271611,-0.073109,-0.084293,0.144371,0.216082,0.100962


In [23]:
X_dev = df[mask]

In [24]:
df[~mask]

,zcr,duration,onset mean,onset std,spect cent mean,spect cent std,0,1,2,3,...,90,91,92,93,94,95,96,97,98,99
Id,,,,,,,,,,,,,,,,,,,,,
9854,-1.168218,-0.467584,-0.318063,-0.113273,-2.101479,-1.411005,0.199921,0.403520,0.192768,0.060535,...,0.420902,0.633787,0.340610,0.042862,-0.038097,0.477749,0.623316,0.094828,0.255115,0.223793
9855,0.227657,0.174802,-0.129621,-0.294424,0.141408,0.489258,0.499998,0.453738,0.431068,0.520585,...,-0.074130,-0.320079,-0.344189,-0.374355,-0.279572,-0.140399,-0.086161,-0.119593,-0.012491,0.009789
9856,-0.183156,-0.253456,-0.026133,-0.120716,-0.331314,-0.605542,0.368010,0.271991,0.246064,0.485611,...,-0.284907,-0.004092,-0.538366,-0.361997,-0.212963,-0.187153,-0.294858,-0.107673,0.289456,0.389897
9857,-0.714797,-1.252723,-1.110443,-1.063533,0.251215,0.671758,0.044053,0.479295,0.475797,0.271325,...,0.187518,0.486476,-0.058100,-0.136523,-0.008058,0.389695,0.095400,0.020955,0.214047,0.262703
9858,-0.160899,-0.396208,-0.484879,-0.261846,0.513179,-0.455979,0.380186,0.544038,0.462708,0.298060,...,-0.027052,0.271499,0.106840,0.153695,-0.026954,-0.154189,-0.185748,-0.147095,-0.330369,-0.368219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11304,0.637198,0.246178,0.412393,0.461266,0.356144,1.705804,0.483181,0.598493,0.373330,0.408912,...,0.121877,0.299779,-0.027250,-0.078064,0.067876,0.174091,-0.102501,0.010891,-0.051230,0.160150
11305,-0.776483,-0.110703,-0.107556,0.251201,-1.021976,-0.922592,0.591019,0.445423,0.450621,0.654377,...,0.056157,0.045312,0.163992,0.098979,0.249748,0.315029,0.203801,0.151159,0.174447,0.334646
11306,-0.624494,-1.324099,-0.901922,-1.059382,1.157546,-0.845783,0.447262,0.612959,0.694631,0.854591,...,-0.053360,0.237588,-0.049829,-0.172128,-0.234809,-0.138374,-0.152203,-0.038802,0.025456,-0.110014


## Random Forest Classifier

In [25]:
from sklearn.metrics import f1_score

In [26]:
def train_rf(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    rfc_prediction = rfc.predict(X_test)
    print(accuracy_score(y_test, rfc_prediction))
   # print(f1_score(y_test,rfc_prediction,average='macro'))
   # print(sorted(zip(X_dev.columns, rfc.feature_importances_), key=lambda x: x[1],reverse=True))
    return rfc_prediction

In [27]:
pred = train_rf(X_dev,y_dev)

0.6763064434297311


In [ ]:
param_grid = {'n_estimators': [50, 100, 300, 500],
              'criterion': ['gini','entropy'],
              'max_depth': [None,50,100,150]
             }
grid_search_RF = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, scoring='accuracy',n_jobs=-1)
grid_search_RF.fit(X_dev, y_dev)
print(grid_search_RF.best_params_)
print(grid_search_RF.best_score_)

In [31]:
#submit Random Forest
rfc = RandomForestClassifier(n_estimators=500,criterion='entropy',max_depth=None)
rfc.fit(X_dev, y_dev)
prediction = rfc.predict(df[~mask])
submit = pd.DataFrame(prediction)
submit = submit.rename_axis('Id')
submit = submit.rename(columns={0: 'Predicted'})
submit.to_csv('submission.csv', index=True)

## SVC Classifier

In [28]:
from sklearn.decomposition import PCA
def train_svc(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    svc = SVC()
    svc.fit(X_train, y_train)
    svc_prediction = svc.predict(X_test)
  #  print(accuracy_score(y_test, svc_prediction))
    print(f1_score(y_test,svc_prediction,average='macro'))

In [29]:
pred = train_svc(X_dev,y_dev)

0.7475034986176194


In [ ]:
param_grid = {'C': [1,2,3,4,5,10, 50,100],
              'gamma': [1, 0.1,0.2],
              'kernel':['rbf','poly']
             }
CV_rfc = GridSearchCV(estimator=SVC(), param_grid=param_grid, scoring='accuracy',n_jobs=-1)
CV_rfc.fit(X_dev, y_dev)
print(CV_rfc.best_params_)
print(CV_rfc.best_score_)

In [32]:
#submit Support Vector Machines
svc = SVC(kernel='rbf',C=5, gamma=0.1)
svc.fit(X_dev, y_dev)
prediction = svc.predict(df[~mask])
submit = pd.DataFrame(prediction)
submit = submit.rename_axis('Id')
submit = submit.rename(columns={0: 'Predicted'})
submit.to_csv('finalsubmission.csv', index=True)